In [1]:
%matplotlib inline
import time
import os
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
# from bing_image_downloader import downloader
import time
import os
import copy

ModuleNotFoundError: No module named 'bing_image_downloader'

## Data Set

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
data_dir = 'data/hymenoptera_data'

# downloader.download("hammer", limit=100,  output_dir='dataset', 
# adult_filter_off=True, force_replace=False, timeout=60)

# downloader.download("nail", limit=100,  output_dir='dataset', 
# adult_filter_off=True, force_replace=False, timeout=60)

## In Memory Data Augmentation

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ]),
}



## Create Data Loaders

In [ ]:


image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataloadersb1 = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=1,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Display Data

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    #mean = np.array([0.485, 0.456, 0.406])
    #std = np.array([0.229, 0.224, 0.225])
    #inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


In [ ]:

# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

In [ ]:
# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

## normalize image data
Pixels in your images are read into the computer memory as 8-bit integers between 0 and 255 for all three color channels.
Neural Networks however need floating point values in a smaller range. 
Ideally, you want pixel values to follow the standard normal distribution with a mean of 0 and a standard deviation of 1.

In [ ]:
mean = torch.zeros(3)
std = torch.zeros(3)
for images, _ in dataloadersb1['train']:
    for i in range(3):
        mean[i]+=images[:,i,:,:].mean()
        std[i]+=images[:,i,:,:].std()
mean.div_(len(dataloadersb1['train']))
std.div_(len(dataloadersb1['train']))

print("Mean:",mean)
print("Std:",std)

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        #rule of thumb is use images about 256x256 for ImageNet-scale networks 
        # or scale to smallest image
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    # transforms.Normalize()
    # subtracts the channel mean and divides by the channel standard deviation.
    # mean: a list of mean values for each channel in the image
    # std: a list of standard deviation values for each channel in the image
        transforms.Normalize(mean=mean, std=std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ]),
}

In [ ]:
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataloadersb1 = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=1,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes


In [ ]:
# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

In [ ]:
# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])